### Примеры и симуляции на Python

#### Симуляция ошибок I/II рода и мощности

In [1]:
import numpy as np
from scipy.stats import norm

def z_test_reject(x, mu0=0.0, sigma=1.0, alpha=0.05):
    n = len(x)
    z = (x.mean() - mu0) / (sigma/np.sqrt(n))
    crit = norm.ppf(1-alpha/2)
    return abs(z) > crit

def estimate_power(mu0=0.0, mu1=0.3, sigma=1.0, n=50, alpha=0.05, B=2000, seed=1):
    rng = np.random.default_rng(seed)
    rejects = 0
    for _ in range(B):
        x = rng.normal(loc=mu1, scale=sigma, size=n)
        if z_test_reject(x, mu0=mu0, sigma=sigma, alpha=alpha):
            rejects += 1
    return rejects / B

print("Estimated power (mu1=0.3, n=50):", estimate_power(mu1=0.3, n=50))

Estimated power (mu1=0.3, n=50): 0.5445


t-test (scipy) и сравнение с permutation test

In [3]:
import numpy as np
from scipy.stats import norm

# Функция для z-теста
def z_test_reject(x, mu0=0.0, sigma=1.0, alpha=0.05):
    """Проверяет, отвергается ли нулевая гипотеза в z-тесте"""
    n = len(x)
    z = (x.mean() - mu0) / (sigma / np.sqrt(n))
    crit = norm.ppf(1 - alpha/2)
    return abs(z) > crit

# Функция для оценки мощности теста
def estimate_power(mu0=0.0, mu1=0.3, sigma=1.0, n=50, alpha=0.05, B=2000, seed=1):
    """Оценивает мощность z-теста при заданной альтернативе"""
    rng = np.random.default_rng(seed)
    rejects = 0
    for _ in range(B):
        x = rng.normal(loc=mu1, scale=sigma, size=n)
        if z_test_reject(x, mu0=mu0, sigma=sigma, alpha=alpha):
            rejects += 1
    return rejects / B

# Оценка мощности
print("Estimated power (mu1=0.3, n=50):", estimate_power(mu1=0.3, n=50))

# Функция для перестановочного теста (permutation test)
def perm_test_mean(a, b, B=5000, seed=42):
    """Перестановочный тест для сравнения средних двух выборок"""
    rng = np.random.default_rng(seed)
    obs = a.mean() - b.mean()  # наблюдаемая разность
    pooled = np.concatenate([a, b])
    count = 0
    for _ in range(B):
        rng.shuffle(pooled)
        new_a = pooled[:len(a)]
        new_b = pooled[len(a):]
        if abs(new_a.mean() - new_b.mean()) >= abs(obs):
            count += 1
    return count / B

# Пример использования перестановочного теста
# Создаем пример данных
np.random.seed(42)
a = np.random.normal(loc=0.0, scale=1.0, size=30)
b = np.random.normal(loc=0.5, scale=1.0, size=30)

print("Permutation p:", perm_test_mean(a, b))

Estimated power (mu1=0.3, n=50): 0.5445
Permutation p: 0.0194


Хи-тест независимости (contingency table)

In [4]:
import numpy as np
from scipy.stats import chi2_contingency

# таблица: строки -- лечение/контроль, столбцы -- успех/неуспех
table = np.array([[30, 70],
                  [20, 80]])
chi2, p, dof, ex = chi2_contingency(table)
print("chi2:", chi2, "p:", p, "expected:", ex)

chi2: 2.16 p: 0.14164469029513255 expected: [[25. 75.]
 [25. 75.]]


Расчет требуемого n:

In [5]:
from scipy.stats import norm
def required_n_z(alpha=0.05, power=0.8, delta=0.3):
    z_a = norm.ppf(1-alpha/2)
    z_b = norm.ppf(power)
    return ((z_a + z_b) / delta)**2

print("n needed (delta=0.3):", int(np.ceil(required_n_z(delta=0.3))))

n needed (delta=0.3): 88


Упражнение 4

In [7]:
import numpy as np
from scipy import stats

# Исходные данные
p_A = 0.10      # конверсия контрольной группы (baseline)
p_B = 0.12      # ожидаемая конверсия экспериментальной группы
alpha = 0.05    # уровень значимости
power = 0.8     # требуемая мощность (1 - β)

print("Исходные параметры:")
print(f"p_A (контроль) = {p_A}")
print(f"p_B (эксперимент) = {p_B}")
print(f"Относительный прирост = {(p_B - p_A)/p_A*100:.1f}%")
print(f"α (уровень значимости) = {alpha}")
print(f"Мощность (1 - β) = {power}")

# Критические значения нормального распределения
z_alpha = stats.norm.ppf(1 - alpha/2)  # для двустороннего теста
z_beta = stats.norm.ppf(power)         # для мощности

print(f"\nКритические значения:")
print(f"z_(α/2) = {z_alpha:.4f} (для α={alpha})")
print(f"z_β = {z_beta:.4f} (для мощности {power})")

# Формула для размера выборки на группу (нормальное приближение)
# Для сравнения двух пропорций, двусторонний тест

# Вариант 1: Используем более простую и распространенную формулу
n_per_group = (
    (z_alpha + z_beta) ** 2 * 
    (p_A * (1 - p_A) + p_B * (1 - p_B)) / 
    (p_B - p_A) ** 2
)

print(f"\nРасчет по формуле:")
print(f"n = (z_α + z_β)² * [p_A(1-p_A) + p_B(1-p_B)] / (p_B - p_A)²")
print(f"  = ({z_alpha:.4f} + {z_beta:.4f})² * [{p_A}·{1-p_A} + {p_B}·{1-p_B}] / ({p_B} - {p_A})²")
print(f"  = {z_alpha+z_beta:.4f}² * [{p_A*(1-p_A)+p_B*(1-p_B):.4f}] / {(p_B-p_A)**2:.6f}")
print(f"  = {(z_alpha+z_beta)**2:.4f} * {p_A*(1-p_A)+p_B*(1-p_B):.4f} / {(p_B-p_A)**2:.6f}")
print(f"  = {n_per_group:.2f}")

# Вариант 2: Более точная формула (если нужна)
pooled_p = (p_A + p_B) / 2
n_per_group_alt = (
    (z_alpha * np.sqrt(2 * pooled_p * (1 - pooled_p)) + 
     z_beta * np.sqrt(p_A * (1 - p_A) + p_B * (1 - p_B))) ** 2 
    / (p_B - p_A) ** 2
)

print(f"\nАльтернативная (более точная) формула:")
print(f"n = {n_per_group_alt:.2f}")

# Используем более точную формулу для окончательного ответа
n_final = n_per_group_alt
print(f"\nИспользуем более точную формулу: n = {n_final:.2f}")

# Округление до тысяч и вывод ответа
n_rounded_thousands = int(np.ceil(n_final / 1000))
print(f"\nОкругление до тысяч:")
print(f"Точное значение: {n_final:.0f}")
print(f"Делим на 1000: {n_final/1000:.2f}")
print(f"Округляем вверх до целых тысяч: {n_rounded_thousands}")

# Проверка мощности для полученного размера выборки
def calculate_power_for_n(n, p_A, p_B, alpha=0.05):
    """Вычисляет мощность для заданного размера выборки"""
    se = np.sqrt(p_A*(1-p_A)/n + p_B*(1-p_B)/n)
    effect = p_B - p_A
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_power = effect / se - z_alpha
    power_calc = stats.norm.cdf(z_power)
    return power_calc

# Проверяем, какая мощность будет для округленного размера
n_test = n_rounded_thousands * 1000
actual_power = calculate_power_for_n(n_test, p_A, p_B, alpha)

print(f"\nПроверка для n = {n_test} на группу:")
print(f"Фактическая мощность: {actual_power:.4f}")
if actual_power >= power:
    print(f"✓ Мощность достаточна (≥ {power})")
else:
    print(f"✗ Мощность недостаточна (< {power})")

# Проверяем минимальный размер, обеспечивающий требуемую мощность
print(f"\nПоиск минимального n, обеспечивающего мощность {power}:")
for n_candidate in [2000, 2500, 3000, 3500, 4000, 4500]:
    pwr = calculate_power_for_n(n_candidate, p_A, p_B, alpha)
    if pwr >= power:
        print(f"n = {n_candidate}: мощность = {pwr:.4f} ✓")
        break
    else:
        print(f"n = {n_candidate}: мощность = {pwr:.4f} ✗")

print(f"\nОТВЕТ: {n_rounded_thousands}")

Исходные параметры:
p_A (контроль) = 0.1
p_B (эксперимент) = 0.12
Относительный прирост = 20.0%
α (уровень значимости) = 0.05
Мощность (1 - β) = 0.8

Критические значения:
z_(α/2) = 1.9600 (для α=0.05)
z_β = 0.8416 (для мощности 0.8)

Расчет по формуле:
n = (z_α + z_β)² * [p_A(1-p_A) + p_B(1-p_B)] / (p_B - p_A)²
  = (1.9600 + 0.8416)² * [0.1·0.9 + 0.12·0.88] / (0.12 - 0.1)²
  = 2.8016² * [0.1956] / 0.000400
  = 7.8489 * 0.1956 / 0.000400
  = 3838.10

Альтернативная (более точная) формула:
n = 3840.85

Используем более точную формулу: n = 3840.85

Округление до тысяч:
Точное значение: 3841
Делим на 1000: 3.84
Округляем вверх до целых тысяч: 4

Проверка для n = 4000 на группу:
Фактическая мощность: 0.8160
✓ Мощность достаточна (≥ 0.8)

Поиск минимального n, обеспечивающего мощность 0.8:
n = 2000: мощность = 0.5249 ✗
n = 2500: мощность = 0.6183 ✗
n = 3000: мощность = 0.6974 ✗
n = 3500: мощность = 0.7628 ✗
n = 4000: мощность = 0.8160 ✓

ОТВЕТ: 4


Упражнение 5

In [11]:
import numpy as np
from scipy.stats import ttest_1samp

# Создаем фиксированный генератор
rng = np.random.default_rng(seed=0)

n, k, alpha = 50, 5, 0.05

# Генерируем данные
data = []
for i in range(n):
    if i < k:
        # С эффектом 0.5
        sample = rng.normal(loc=0.5, scale=1, size=30)
    else:
        # Без эффекта
        sample = rng.normal(loc=0, scale=1, size=30)
    data.append(sample)

# Проводим t-тесты
false_positives = 0
for i in range(k, n):
    t_stat, p_value = ttest_1samp(data[i], popmean=0)
    if p_value < alpha:
        false_positives += 1

print(f"Ложноположительных результатов: {false_positives}")

Ложноположительных результатов: 2


### Одновыборочная гауссовская модель. Проверка гипотез

In [12]:
# Вариант, если сигма известно

import numpy as np
from scipy.stats import norm, t, ttest_1samp

n=25; xbar=5.2; sigma=1.1; mu0=5.0
z = (xbar-mu0)/(sigma/np.sqrt(n))
p_two = 2*(1-norm.cdf(abs(z)))
z, p_two

(np.float64(0.9090909090909097), np.float64(0.3633021408868977))

Если сигма неизвестна,  используем  ttest_1samp или формулу для T

### Биномиальная модель

In [15]:
from scipy.stats import binomtest
k = 5
n = 20
p0 = 0.3
res = binomtest(k, n, p=p0, alternative='two-sided')
res.pvalue

np.float64(0.8083610172465565)

### Надежные интервалы и тесты для долей

In [ ]:
#Пример: 
    
import statsmodels.stats.proportion as smp
smp.proportion_confint(count=k, nobs=n, alpha=0.05, method='wilson')
smp.proportion_confint(count=k, nobs=n, alpha=0.05, method='beta')  # Clopper-Pearson

### Примеры

Рассмотрим для Z и t:

In [17]:
import numpy as np
from scipy.stats import norm, t, ttest_1samp

# пример: sigma известно
n=50; xbar=125.8; sigma=10
z = (xbar-125)/(sigma/np.sqrt(n))
p_two = 2*(1-norm.cdf(abs(z)))

# пример: sigma неизвестен
data = np.random.normal(loc=0.3, scale=1.2, size=25)
tstat, pval = ttest_1samp(data, popmean=0.0)

Биномиальный тест и CI:

In [18]:
import numpy as np
from scipy.stats import norm, t, ttest_1samp

# пример: sigma известно
n=50; xbar=125.8; sigma=10
z = (xbar-125)/(sigma/np.sqrt(n))
p_two = 2*(1-norm.cdf(abs(z)))

# пример: sigma неизвестен
data = np.random.normal(loc=0.3, scale=1.2, size=25)
tstat, pval = ttest_1samp(data, popmean=0.0)

Упражнение 1

In [20]:
# пример: sigma известно
n=30; xbar=0; sigma=1
z = (xbar-0.4)/(sigma/np.sqrt(n))
p_two = 2*(1-norm.cdf(abs(z)))
p_two

np.float64(0.02845973691631065)

Упражнение 2

In [21]:
import numpy as np
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import binomtest

# Данные
n = 40
k = 3
p0 = 0.1
alpha = 0.05

# 1. Точный биномиальный тест (binomtest)
exact_test = binomtest(k, n, p0, alternative='two-sided')
exact_p = exact_test.pvalue

# 2. Wald CI
p_hat = k / n
z = 1.96  # для 95% ДИ
wald_se = np.sqrt(p_hat * (1 - p_hat) / n)
wald_ci = (p_hat - z * wald_se, p_hat + z * wald_se)

# 3. Wilson CI (score interval)
wilson_ci = proportion_confint(k, n, alpha=alpha, method='wilson')

print(f"Данные: {k} успехов из {n} (p̂ = {p_hat:.3f})")
print(f"\n1. Точный биномиальный тест (H₀: p = {p0}):")
print(f"   p-value = {exact_p:.4f}")

print(f"\n2. Wald доверительный интервал (95%):")
print(f"   ({wald_ci[0]:.4f}, {wald_ci[1]:.4f})")

print(f"\n3. Wilson доверительный интервал (95%):")
print(f"   ({wilson_ci[0]:.4f}, {wilson_ci[1]:.4f})")

# Проверка: попадает ли p0 в доверительные интервалы?
in_wald = wald_ci[0] <= p0 <= wald_ci[1]
in_wilson = wilson_ci[0] <= p0 <= wilson_ci[1]

print(f"\nПроверка H₀ (p = {p0}) по доверительным интервалам:")
print(f"   Wald: {'Не отвергаем' if in_wald else 'Отвергаем'} (p₀ в ДИ: {in_wald})")
print(f"   Wilson: {'Не отвергаем' if in_wilson else 'Отвергаем'} (p₀ в ДИ: {in_wilson})")
print(f"   Точный тест: {'Не отвергаем' if exact_p > alpha else 'Отвергаем'} (p-value = {exact_p:.4f})")

print(f"\nВЫВОД: Для маленькой выборки и редких событий надежнее всего ТОЧНЫЙ БИНОМИАЛЬНЫЙ ТЕСТ")

Данные: 3 успехов из 40 (p̂ = 0.075)

1. Точный биномиальный тест (H₀: p = 0.1):
   p-value = 0.7941

2. Wald доверительный интервал (95%):
   (-0.0066, 0.1566)

3. Wilson доверительный интервал (95%):
   (0.0258, 0.1986)

Проверка H₀ (p = 0.1) по доверительным интервалам:
   Wald: Не отвергаем (p₀ в ДИ: True)
   Wilson: Не отвергаем (p₀ в ДИ: True)
   Точный тест: Не отвергаем (p-value = 0.7941)

ВЫВОД: Для маленькой выборки и редких событий надежнее всего ТОЧНЫЙ БИНОМИАЛЬНЫЙ ТЕСТ


Упражнение 3

In [22]:
import numpy as np
from scipy import stats

# Параметры
p_A = 0.10
p_B = 0.12
alpha = 0.05
power = 0.8

# Критические значения
z_alpha = stats.norm.ppf(1 - alpha/2)  # 1.96
z_beta = stats.norm.ppf(power)         # 0.8416

# Формула для размера выборки на группу
n = ((z_alpha + z_beta)**2 * (p_A*(1-p_A) + p_B*(1-p_B))) / ((p_B - p_A)**2)

print(f"Точное n: {n:.0f}")
print(f"Округлено до тысяч: {int(np.ceil(n/1000))}")

Точное n: 3838
Округлено до тысяч: 4


Упражнение 4

In [23]:
import numpy as np
from scipy import stats

np.random.seed(42)
data = np.random.lognormal(0, 1, 30)

# t-CI
t_ci = stats.t.interval(0.95, len(data)-1, loc=np.mean(data), 
                        scale=stats.sem(data))

# Bootstrap
B = 5000
idx = np.random.randint(0, len(data), size=(B, len(data)))
boot_means = np.mean(data[idx], axis=1)
boot_ci = np.percentile(boot_means, [2.5, 97.5])

print(f"t-CI: ({t_ci[0]:.3f}, {t_ci[1]:.3f})")
print(f"Bootstrap: ({boot_ci[0]:.3f}, {boot_ci[1]:.3f})")

t-CI: (0.763, 1.700)
Bootstrap: (0.829, 1.709)


Упражнение 5

In [ ]:
import numpy as np
from scipy import stats

# Данные
n = 20
x_bar = 5.6
s = 1.0
mu0 = 5.0
alpha = 0.05

# Односторонний t-test (правосторонний)
t_stat = (x_bar - mu0) / (s / np.sqrt(n))
p_value = 1 - stats.t.cdf(t_stat, df=n-1)

# Односторонний 95% CI (нижняя граница)
t_crit = stats.t.ppf(1 - alpha, df=n-1)  # для одностороннего
lower_bound = x_bar - t_crit * (s / np.sqrt(n))

print(f"t-статистика: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Односторонний 95% CI (нижняя граница): {lower_bound:.3f}")
print(f"Округлено до целого: {int(np.round(lower_bound))}")

t-статистика: 2.683
p-value: 0.0074
Односторонний 95% CI (нижняя граница): 5.213
Округлено до целого: 5


: 